In [1]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src', 'data')
sys.path.append(src_dir)

In [2]:
%aimport scrape_buda

In [3]:
from scrape_buda import BudaRating

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
import pymysql as mdb

con = mdb.connect('localhost', 'budauser', 'TrotNixon', 'buda_prod') #host, user, password, 

In [7]:
sqlfile = os.path.join(os.getcwd(), os.pardir, 'data', 'raw', 'buda_prod_backup_2016-08-31.sql')

In [8]:
interim_dir = os.path.join(os.getcwd(), os.pardir, 'data', 'interim')

In [9]:
sqlfile = """SELECT * FROM buda_users_public;"""
buda_users_public = pd.read_sql(sqlfile, con, index_col='user_id')

In [12]:
sqlfile = """SELECT * FROM buda_hatleague_player_rankings;"""
buda_hatleague_player_rankings = pd.read_sql(sqlfile, con)

In [13]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.rename(columns={'player_id':'user_id'})

In [14]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.set_index('user_id')

In [15]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.drop('edited_date', axis=1)

In [16]:
self_ratings = buda_hatleague_player_rankings.join(buda_users_public)

In [65]:
self_ratings[(self_ratings['last_name'] == 'Johnson') & (self_ratings['first_name'] == 'Kyle')]

,league_id,rank_type,status,edited_by,modification_id,rank,experience,other_sport,throwing,break_mark,huck,receiving,height,speed,shirt,recommend_captain,user_name,email,password,first_name,last_name,edited_date,last_login
user_id,,,,,,,,,,,,,,,,,,,,,,,
45938,0,1,NaN,45938,None,65.0,div2,1.0,reliable,sometime,short,leaping,72.0,average,large,0.0,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,36881,1,NaN,3424,None,59.0,None,NaN,None,None,None,None,NaN,None,None,NaN,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,37648,1,NaN,24685,None,59.0,None,NaN,None,None,None,None,NaN,None,None,NaN,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,37648,2,NaN,28926,None,50.0,div2,1.0,reliable,none,none,one_hand,0.0,fast,,1.0,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,38484,1,NaN,29236,None,55.0,None,NaN,None,None,None,None,NaN,None,None,NaN,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,38484,2,NaN,25532,None,35.0,hat,1.0,reliable,none,none,both_hands,0.0,average,,0.0,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,39622,1,NaN,29236,None,54.0,None,NaN,None,None,None,None,NaN,None,None,NaN,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,39622,2,NaN,25532,None,55.0,div2,1.0,reliable,none,none,leaping,0.0,average,,0.0,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12
45938,40258,1,NaN,29236,None,56.0,None,NaN,None,None,None,None,NaN,None,None,NaN,engagekage,engagekage1988@aol.com,cd90aeb927b5dd3fef207d62bf2c982f:v1V9rXS2mh0px...,Kyle,Johnson,2013-04-30 01:49:32,2016-08-24 19:40:12


In [62]:
hmm = self_ratings.replace('nan', -1)

In [64]:
len(hmm[hmm['rank'] == -1])

552

In [29]:
sqlfile = """SELECT league_id,league_name,season FROM buda_leagues;"""
buda_leagues = pd.read_sql(sqlfile, con)

In [36]:
self_ratings_league = self_ratings.reset_index().merge(buda_leagues, on='league_id')

In [37]:
len(self_ratings)

47921

In [38]:
len(self_ratings_league)

38296

In [39]:
self_ratings_league = self_ratings_league.drop(['modification_id', 'user_name', 'email', 'password'], axis=1)

In [43]:
self_ratings_league.to_csv(os.path.join(interim_dir, 'self_ratings.csv'))

In [68]:
self_ratings_league[(self_ratings_league['first_name'] == 'Martin')].sort('league_id')

,user_id,league_id,rank_type,status,edited_by,rank,experience,other_sport,throwing,break_mark,huck,receiving,height,speed,shirt,recommend_captain,first_name,last_name,edited_date,last_login,league_name,season
10863,651,5129,1,NaN,164,5.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,De lasa,2003-05-22 21:55:42,2003-05-22 21:55:42,Summer Hat League,2002
11387,6278,6235,1,NaN,2495,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Bakal,2005-03-24 20:13:50,2014-10-23 15:54:15,Winter (Indoor) Hat League,2003
394,6278,6326,1,0.0,3424,53.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Bakal,2005-03-24 20:13:50,2014-10-23 15:54:15,Spring Hat League,2003
11706,9093,6340,1,NaN,3424,59.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Rotemberg,2005-05-29 14:27:58,2005-06-05 20:36:25,Summer Hat League,2003
11430,651,6340,1,NaN,3424,70.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,De lasa,2003-05-22 21:55:42,2003-05-22 21:55:42,Summer Hat League,2003
840,6278,10918,1,NaN,3424,53.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Bakal,2005-03-24 20:13:50,2014-10-23 15:54:15,Fall Hat League,2003
1630,11650,11402,1,NaN,3424,42.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Scherstuhl,2004-03-29 22:24:46,2004-03-29 22:24:46,Spring Hat League,2004
16422,9093,11604,1,NaN,3424,60.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Rotemberg,2005-05-29 14:27:58,2005-06-05 20:36:25,Summer Hat League,2004
2805,6278,13445,1,NaN,3424,53.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Bakal,2005-03-24 20:13:50,2014-10-23 15:54:15,Spring Hat League,2005
3235,13653,13445,1,NaN,3424,39.0,None,NaN,None,None,None,None,NaN,None,None,NaN,Martin,Minns,2008-03-24 20:41:45,2016-03-09 16:22:48,Spring Hat League,2005


In [51]:
ratings.team_players['40328'][2].split(',')[1][1:]

u'Shane'

In [54]:
for league_id in ratings.league_teams:
    if '40328' in ratings.league_teams[league_id]:
        print(league_id)

40258
